In [2]:
from termcolor import colored
from pyparsing import nestedExpr
from IPython.display import clear_output 
import requests
from tensorflow import keras
import os.path
from os import path
import zipfile


tubes = []
numberOfTubes = 0
stage = 0
autoSolveActive = False

class DFS:
  
  class Node:
    def __init__(self, tubes, parent=None, gscore=0, fromT= None, toT= None):
      self.tubes = tubes
      self.parent = parent
      self.gscore = gscore
      self.fromT = fromT
      self.toT = toT


    def __eq__(self, other):
      if other == None:
        return False
      return other.tubes == self.tubes

    def __lt__(self, other):
      return self.gscore < other.gscore
      
  def __init__(self,tubes):
    self.tubes = tubes
    self.startNode = self.Node(self.tubes)
    self.nodeSeen = []
    self.nodeQueue = []
    self.stage = 0
    

  def getChildren(self, node):
    children = []
    if self.available(node):
      tubesNew = []
      for tube in node.tubes:
        tubesNew.append(tube.copy())
      for fromTube in range(len(tubesNew)):
        for toTube in range(len(tubesNew)):
          if validPour(tubesNew,fromTube, toTube):
            newNode = self.Node(pour(tubesNew,fromTube, toTube),node,node.gscore - 1,fromT = fromTube,toT = toTube)
            children.append (newNode)
    return children
  def contains(self,nodeList,node):
    contains = False
    if node == None:
      return False
    for item in nodeList:
      if item == None:
        continue
      if item == node:
        contains = True
        break
    return contains

  def findpath(self,tubes):
    mainNode = self.Node(tubes)
    if isFinished(mainNode.tubes):
      return mainNode
    for node in self.getChildren(mainNode):
      if self.contains(self.nodeSeen,node):
        continue
      else:
        self.nodeSeen.append(node)
        self.nodeQueue.append(node)
    self.nodeQueue.sort()
    notFinished = True
    while notFinished:
      newNode = self.nodeQueue.pop(0)
      if isFinished(newNode.tubes):
        return newNode
      for nodeChild in self.getChildren(newNode):
        if self.contains(self.nodeSeen,nodeChild):
          continue
        else:
          self.nodeSeen.append(nodeChild)
          self.nodeQueue.append(nodeChild)
      self.nodeQueue.sort()
    return None

  def available(self, node):
    if node.tubes != None:
      return True
    return False

  def generate(self, node):
    path = list()
    while node:
      path.append(node)
      node = node.parent
    return path



class UCS:
  class Node:
    def __init__(self, tubes, parent=None, gscore=0, fromT= None, toT= None):
      self.tubes = tubes
      self.parent = parent
      self.gscore = gscore
      self.fromT = fromT
      self.toT = toT


    def __eq__(self, other):
      if other == None:
        return False
      return other.tubes == self.tubes

    def __lt__(self, other):
      return self.gscore  < other.gscore 
      
  def __init__(self,tubes):
    self.tubes = tubes
    self.startNode = self.Node(self.tubes)
    self.nodeSeen = []
    self.nodeQueue = []
    self.stage = 0

  def getChildren(self, node):
    children = []
    if self.available(node):
      tubesNew = []
      for tube in node.tubes:
        tubesNew.append(tube.copy())
      for fromTube in range(len(tubesNew)):
        for toTube in range(len(tubesNew)):
          if validPour(tubesNew,fromTube, toTube):
            newNode = self.Node(pour(tubesNew,fromTube, toTube),node,node.gscore + 1,fromT = fromTube,toT = toTube)
            children.append (newNode)
    return children
  def contains(self,nodeList,node):
    contains = False
    if node == None:
      return False
    for item in nodeList:
      if item == None:
        continue
      if item == node:
        contains = True
        break
    return contains

  def findpath(self,tubes):
    mainNode = self.Node(tubes)
    if isFinished(mainNode.tubes):
      return mainNode
    for node in self.getChildren(mainNode):
      if self.contains(self.nodeSeen,node):
        continue
      else:
        self.nodeSeen.append(node)
        self.nodeQueue.append(node)
    self.nodeQueue.sort()
    notFinished = True
    while notFinished:
      newNode = self.nodeQueue.pop(0)
      if isFinished(newNode.tubes):
        return newNode
      for nodeChild in self.getChildren(newNode):
        if self.contains(self.nodeSeen,nodeChild):
          continue
        else:
          self.nodeSeen.append(nodeChild)
          self.nodeQueue.append(nodeChild)
      self.nodeQueue.sort()
    return None

  def available(self, node):
    if node.tubes != None:
      return True
    return False

  def generate(self, node):
    path = list()
    while node:
      path.append(node)
      node = node.parent
    return path
    
class Greedy:
  class Node:
    def __init__(self, tubes, parent=None, heuristic=100000, fromT= None, toT= None):
      self.tubes = tubes
      self.parent = parent
      self.heuristic = heuristic
      self.fromT = fromT
      self.toT = toT

    def __eq__(self, other):
      if other == None:
        return False
      return other.tubes == self.tubes

    def __lt__(self, other):
      return self.heuristic <  other.heuristic
      
  def __init__(self,tubes):
    self.tubes = tubes
    self.startNode = self.Node(self.tubes)
    self.nodeSeen = []
    self.nodeQueue = []
    self.stage = 0
    
  def heuristic(self, node):
    sum = 0
    for tube in node.tubes:
      eachTube = 0
      j = 0
      top = ""
      isTopSet = False
      for i, value in enumerate(tube):
        if value == "":
          continue
        elif not isTopSet:
          isTopSet = True
          top = value
          continue
        elif value == top:
          j +=1
          eachTube += j * 100
        else:
          break
      sum += eachTube
    return sum * (-1)

  def getChildren(self, node):
    children = []
    if self.available(node):
      tubesNew = []
      for tube in node.tubes:
        tubesNew.append(tube.copy())
      for fromTube in range(len(tubesNew)):
        for toTube in range(len(tubesNew)):
          if validPour(tubesNew,fromTube, toTube):
            newNode = self.Node(pour(tubesNew,fromTube, toTube),node,fromT = fromTube,toT = toTube)
            newNode.heuristic = self.heuristic(node)
            children.append (newNode)
    return children
  def contains(self,nodeList,node):
    contains = False
    if node == None:
      return False
    for item in nodeList:
      if item == None:
        continue
      if item == node:
        contains = True
        break
    return contains

  def findpath(self,tubes):
    mainNode = self.Node(tubes)
    if isFinished(mainNode.tubes):
      return mainNode
    for node in self.getChildren(mainNode):
      if self.contains(self.nodeSeen,node):
        continue
      else:
        self.nodeSeen.append(node)
        self.nodeQueue.append(node)
    self.nodeQueue.sort()
    notFinished = True
    while notFinished:
      newNode = self.nodeQueue.pop(0)
      if isFinished(newNode.tubes):
        return newNode
      for nodeChild in self.getChildren(newNode):
        if self.contains(self.nodeSeen,nodeChild):
          continue
        else:
          self.nodeSeen.append(nodeChild)
          self.nodeQueue.append(nodeChild)
      self.nodeQueue.sort()
    return None

  def available(self, node):
    if node.tubes != None:
      return True
    return False

  def generate(self, node):
    path = list()
    while node:
      path.append(node)
      node = node.parent
    return path

class AStar:
  class Node:
    def __init__(self, tubes, parent=None, gscore=0, heuristic=100000, fromT= None, toT= None):
      self.tubes = tubes
      self.parent = parent
      self.gscore = gscore
      self.heuristic = heuristic
      self.fromT = fromT
      self.toT = toT


    def __eq__(self, other):
      if other == None:
        return False
      return other.tubes == self.tubes

    def __lt__(self, other):
      return self.gscore + self.heuristic < other.gscore + other.heuristic
      
  def __init__(self,tubes):
    self.tubes = tubes
    self.startNode = self.Node(self.tubes)
    self.nodeSeen = []
    self.nodeQueue = []
    self.stage = 0
    
  def heuristic(self, node):
    sum = 0
    for tube in node.tubes:
      eachTube = 0
      j = 0
      top = ""
      isTopSet = False
      for i, value in enumerate(tube):
        if value == "":
          continue
        elif not isTopSet:
          isTopSet = True
          top = value
          continue
        elif value == top:
          j +=1
          eachTube += j * 100
        else:
          break
      sum += eachTube
    return sum * (-1)

  def getChildren(self, node):
    children = []
    if self.available(node):
      tubesNew = []
      for tube in node.tubes:
        tubesNew.append(tube.copy())
      for fromTube in range(len(tubesNew)):
        for toTube in range(len(tubesNew)):
          if validPour(tubesNew,fromTube, toTube):
            newNode = self.Node(pour(tubesNew,fromTube, toTube),node,node.gscore + 1,fromT = fromTube,toT = toTube)
            newNode.heuristic = self.heuristic(node)
            children.append (newNode)
    return children
  def contains(self,nodeList,node):
    contains = False
    if node == None:
      return False
    for item in nodeList:
      if item == None:
        continue
      if item == node:
        contains = True
        break
    return contains

  def findpath(self,tubes):
    mainNode = self.Node(tubes)
    if isFinished(mainNode.tubes):
      return mainNode
    for node in self.getChildren(mainNode):
      if self.contains(self.nodeSeen,node):
        continue
      else:
        self.nodeSeen.append(node)
        self.nodeQueue.append(node)
    self.nodeQueue.sort()
    notFinished = True
    while notFinished:
      newNode = self.nodeQueue.pop(0)
      if isFinished(newNode.tubes):
        return newNode
      for nodeChild in self.getChildren(newNode):
        if self.contains(self.nodeSeen,nodeChild):
          continue
        else:
          self.nodeSeen.append(nodeChild)
          self.nodeQueue.append(nodeChild)
      self.nodeQueue.sort()
    return None

  def available(self, node):
    if node.tubes != None:
      return True
    return False

  def generate(self, node):
    path = list()
    while node:
      path.append(node)
      node = node.parent
    return path

def validColor(color):
  colors = ["","grey","red","green","yellow","blue","magenta","cyan","white"]
  if color in colors:
    return True
  else:
    return False
def colorChar(color):
  text = colored("\u2588",color)
  #text = colored("#",color)
  print(text,end="")

def printOneTube(color1,color2,color3,color4):
  if color1 == "" :
    print("| |")
  else:
      print("|",end="");colorChar(color1);print("|")
  if color2 == "" :
    print("| |")
  else:
    print("|",end="");colorChar(color2);print("|")
  if color3 == "" :
    print("| |")
  else:
    print("|",end="");colorChar(color3);print("|")
  if color4 == "" :
    print("| |")
  else:
    print("|",end="");colorChar(color4);print("|")
  print(" ‾ ")

def printTubes(tubes,fromTube, toTube):
  global autoSolveActive
  number = 0
  print()
  if autoSolveActive:
    print("########################")
    print("# Auto-Solve activated #")
    print("########################")
    print()
  if fromTube != -1 and toTube != -1:
    print()
    print()
    print()
    print()
    print("Stage "+str(stage))
    print("Poured from "+str(fromTube+1)+" to "+str(toTube+1))
    print(str(fromTube+1)+" -> "+str(toTube+1))
  print("##############################################################################")
  for tube in tubes:
    number+=1
    print("Tube No. "+str(number)+":");printOneTube(tube[0],tube[1],tube[2],tube[3])
  print("##############################################################################")

def validPour(tubes,fromTube, toTube):
  fromTubeTop = ""
  toTubeTop = ""
  if tubes[fromTube][0] != "":
    fromTubeTop = tubes[fromTube][0]
  elif tubes[fromTube][1] != "":
    fromTubeTop = tubes[fromTube][1]
  elif tubes[fromTube][2] != "":
    fromTubeTop = tubes[fromTube][2]
  elif tubes[fromTube][3] != "":
    fromTubeTop = tubes[fromTube][3]
  else:
    fromTubeTop = ""
  
  if tubes[toTube][0] != "":
    toTubeTop = tubes[toTube][0]
  elif tubes[toTube][1] != "":
    toTubeTop = tubes[toTube][1]
  elif tubes[toTube][2] != "":
    toTubeTop = tubes[toTube][2]
  elif tubes[toTube][3] != "":
    toTubeTop = tubes[toTube][3]
  else:
    toTubeTop = ""
  
  if fromTubeTop == "":
    return False
  if fromTubeTop != toTubeTop and toTubeTop != "":
    return False
  if tubes[toTube][0] != "":
    return False
  if fromTube == toTube:
    return False
  return True
  

def pour(tubesRaw, fromTube,toTube):
  tubesNew = list()
  for tube in tubesRaw:
    tubesNew.append(tube.copy())
  fromTubeTop = ""
  fromTubeTopIndex = 0
  toTubeTop = ""
  toTubeTopIndex = 0

  if tubesNew[fromTube][0] != "":
    fromTubeTop = tubesNew[fromTube][0]
    fromTubeTopIndex = 0
  elif tubesNew[fromTube][1] != "":
    fromTubeTop = tubesNew[fromTube][1]
    fromTubeTopIndex = 1
  elif tubesNew[fromTube][2] != "":
    fromTubeTop = tubesNew[fromTube][2]
    fromTubeTopIndex = 2
  elif tubesNew[fromTube][3] != "":
    fromTubeTop = tubesNew[fromTube][3]
    fromTubeTopIndex = 3
  else:
    fromTubeTop = ""
    fromTubeTopIndex = -1
  
  if tubesNew[toTube][0] != "":
    toTubeTop = tubesNew[toTube][0]
    toTubeTopIndex = -1
  elif tubesNew[toTube][1] != "":
    toTubeTop = tubesNew[toTube][1]
    toTubeTopIndex = 0
  elif tubesNew[toTube][2] != "":
    toTubeTop = tubesNew[toTube][2]
    toTubeTopIndex = 1
  elif tubesNew[toTube][3] != "":
    toTubeTop = tubesNew[toTube][3]
    toTubeTopIndex = 2
  else:
    toTubeTop = ""
    toTubeTopIndex = 3
  if fromTubeTopIndex == -1 or toTubeTopIndex == -1:
    return False
  else:
    tubesNew[fromTube][fromTubeTopIndex] = ""
    tubesNew[toTube][toTubeTopIndex] = fromTubeTop
    fromTubeTopIndex += 1
    if fromTubeTopIndex < 4:
      if tubesNew[fromTube][fromTubeTopIndex] == fromTubeTop:
        if validPour(tubesNew,fromTube,toTube):
          tubesNew = pour(tubesNew,fromTube,toTube)
  return tubesNew
  #clear_output()
  
def autoSolver():
  global autoSolveActive
  global tubes
  autoSolveActive = True
  global stage
  stage -=1

  # A* Algorithm Start
  tubesNew = list()
  for tube in tubes:
    tubesNew.append(tube.copy())
  AStarStage  = -1
  a_star = AStar(tubesNew)
  lastNode = a_star.findpath(a_star.tubes)
  if lastNode != None:
    path = a_star.generate(lastNode)
    for nodePath in path[::-1]:
      AStarStage += 1
      stage +=1
      if nodePath == None:
        continue
      if (nodePath.toT == None or nodePath.fromT == None):
        printTubes(nodePath.tubes,-1,-1)
      else:
        printTubes(nodePath.tubes,nodePath.fromT,nodePath.toT)
    a_star.stage = AStarStage
  else:
    print("Not Solvable by A*!!!!!!")
    return False

  # A* Algorithm End
  print("A* algorithm: Number of stages:["+str(a_star.stage)+"], Number of expanded nodes:["+str(len(a_star.nodeSeen))+"]")

  # Greedy Algorithm Start
  tubesNew = list()
  for tube in tubes:
    tubesNew.append(tube.copy())
  greedyStage  = -1

  greedy = Greedy(tubesNew)
  lastNode = greedy.findpath(greedy.tubes)
  if lastNode != None:
    path = greedy.generate(lastNode)
    for nodePath in path[::-1]:
      greedyStage += 1
    greedy.stage = greedyStage
  else:
    print("Not Solvable by Greedy!!!!!!")
    return False
  # Greedy Algorithm End
  print("Greedy algorithm: Number of stages:["+str(greedy.stage)+"], Number of expanded nodes:["+str(len(greedy.nodeSeen))+"]")

  # UCS Algorithm Start
  tubesNew = list()
  for tube in tubes:
    tubesNew.append(tube.copy())
  ucsStage  = -1

  ucs = UCS(tubesNew)
  lastNode = ucs.findpath(ucs.tubes)
  if lastNode != None:
    path = ucs.generate(lastNode)
    for nodePath in path[::-1]:
      ucsStage += 1
    ucs.stage = ucsStage
  else:
    print("Not Solvable by UCS!!!!!!")
    return False
  # UCS Algorithm End
  print("UCS algorithm: Number of stages:["+str(ucs.stage)+"], Number of expanded nodes:["+str(len(ucs.nodeSeen))+"]")

  # DFS Algorithm Start
  tubesNew = list()
  for tube in tubes:
    tubesNew.append(tube.copy())
  dfsStage  = -1

  dfs = DFS(tubesNew)
  lastNode = dfs.findpath(dfs.tubes)
  if lastNode != None:
    path = dfs.generate(lastNode)
    for nodePath in path[::-1]:
      dfsStage += 1
    dfs.stage = dfsStage
  else:
    print("Not Solvable by DFS!!!!!!")
    return False
  # DFS Algorithm End
  print("DFS algorithm: Number of stages:["+str(dfs.stage)+"], Number of expanded nodes:["+str(len(dfs.nodeSeen))+"]")

  

def isEmpty(tube):
  empty = True
  for item in tube:
    if item != "":
      empty = False
  return empty

def isAllOneColor(tube):
  oneColor = True
  item0 = tube[0]
  for item in tube:
    if item != item0:
      oneColor =False
  return oneColor

def isFinished(tubesList):
  isFinished = True
  for tube in tubesList:
    if not isAllOneColor(tube):
      isFinished =False
  return isFinished    

def showFinished():
  text = colored("############","green")
  print (text)
  text = colored("# Finished #","green")
  print (text)
  text = colored("############","green")
  print (text)

def readFile(level):
  if path.exists('/media/Levels') == False:
    os.mkdir('/media/Levels')

  dataset_url = "https://drive.google.com/uc?id=1-47B019O3zIt7F2-L4BKwXU6Dckuw0ll&export=download"
  data_dir = keras.utils.get_file('/media/LevelZip.zip', origin=dataset_url)

  with zipfile.ZipFile("/media/LevelZip.zip", 'r') as zip_ref:
    zip_ref.extractall("/media/Levels")

  global numberOfTubes
  fileName = "/media/Levels/"+level+".txt"
  with open(fileName) as f:
    lines = f.readlines()
  for line in lines:
    temp = []
    text = nestedExpr('{','}').parseString(line).asList()
    for items in text:
      for item in items:
        item = "".join(item)
        if validColor(item):
          temp.append(item)
        else:
          print("wrong item: "+item)
          return False
    tubes.append(temp)
    numberOfTubes = numberOfTubes+ 1
  return True

level = input("Enter the level number: ")
isWrongColor= False
if not readFile(level):
  print("Wrong color or wrong Fromat!!!")
  isWrongColor =True
  exit()
printTubes(tubes,-1,-1)
while(True):
  if isWrongColor:
    break
  if isFinished(tubes):
    showFinished()
    break
  print("Enter \'X\' for exit and \'A\' for auto-solve")
  fromTube = input("Pour from tube: ")
  if fromTube.lower()== 'x':
    break
  elif fromTube.lower() == 'a':
    autoSolver()
    break
  else:
    fromTube = int(fromTube)
    fromTube-=1
    toTube = int (input("To tube: "))
    toTube-=1
    if validPour(tubes, fromTube,toTube):
      tubes = pour(tubes,fromTube,toTube)
      stage +=1
      printTubes(tubes,fromTube,toTube)
      #print(tubes)
    else:
      print("Not a valid pour! Try again")
    


Enter the level number: 3

##############################################################################
Tube No. 1:
|█|
|█|
|█|
|█|
 ‾ 
Tube No. 2:
|█|
|█|
|█|
|█|
 ‾ 
Tube No. 3:
|█|
|█|
|█|
|█|
 ‾ 
Tube No. 4:
| |
| |
| |
| |
 ‾ 
Tube No. 5:
| |
| |
| |
| |
 ‾ 
##############################################################################
Enter 'X' for exit and 'A' for auto-solve
Pour from tube: a

########################
# Auto-Solve activated #
########################

##############################################################################
Tube No. 1:
|█|
|█|
|█|
|█|
 ‾ 
Tube No. 2:
|█|
|█|
|█|
|█|
 ‾ 
Tube No. 3:
|█|
|█|
|█|
|█|
 ‾ 
Tube No. 4:
| |
| |
| |
| |
 ‾ 
Tube No. 5:
| |
| |
| |
| |
 ‾ 
##############################################################################

########################
# Auto-Solve activated #
########################





Stage 1
Poured from 1 to 4
1 -> 4
##############################################################################
Tube N

#DFS 
Every g(n) is reducing one by one per generation meaning chilren have more priority than their uncles

Rest of the funcions same as BFS and UCS

#UCS (Same as BFS) 
Because g(n) is adding one by one, it means parents have more priority than children. So every generation is seen then we move to next generation

Node class is a class that we use every instance of it as a state in our search

getChildren() Finds all children of the parent and their g(n) is one unit more than the parent so the child is searched after previous generation.

contains() Checks if the list conatins a node.

findpath() find the goal state from the starting state that is given to it.

available() checks if this node is available or not

generate() generats the path to the found goal from the starting point. And adding them all to a list to iterate.


#Greedy (Also A*)
Because for A*, g(n) = 1 for each step, which doesn't make a difference.

Same functions as A* but doesn't have a g(n) function

#A*
Node class is a class that we use every instance of it as a state in our search

heuristic() It calculate how many of tubes have a sequence more than one color on top for example: [red,red,green,blue] has a heuristic value of 1 * 100 (1 is because of two reds on top) and [green,green,green,blue] has a heuristic of 1* 100 + 2 * 100.

getChildren() Finds all children of the parent by checking all possible but not repeated states that can be generated by this state

contains() Checks if the list conatins a node.

findpath() find the goal state from the starting state that is given to it.

available() checks if this node is available or not

generate() generats the path to the found goal from the starting point. And adding them all to a list to iterate.

g(n) is calculated as each child has one more than it's parent

# Functions:

validColor(): Check if color is valid, meaning is it in the list or not.

colorChar(): Coloring each block in a tube.

printOneTube(): Printing a tube based one colors inputed as parameter.

printTubes(): Printing all tubes using previous function.

validPour(): Checking if this pour (from this tube to that tube) is valid or not.

pour(): Pouring from one tube to another. Usually it is called after previous funcion.

autoSolver(): By calling this, the auto searches begin to search; A*, greedy, UCS (same as BFS), DFS and they try to find the state that all tubes have a unique color. The stages where A* searches is shown. At the end, stage and number of nodes that expanded by each one of the searches are shown.

isFinished(): Checks if the final state is here or not. By calling function isOneColor() it can tell that each tube is having a unique color or not, meaning that game is finished.

readFile(): Downloads and reads file.

User can exit by pressing 'x' and can be activated by pressing 'a'